## JSON 序列化
在不同的编程语言之间传递对象，必须把对象序列化为标准格式，比较好的方法是序列化为 `JSON`。

因为 `JSON` 表示出来就是一个字符串，可以被所有语言读取，也可以方便地存储到磁盘或者通过网络传输。

### JSON类型

`JSON` 和 Python 内置的数据类型对应如下：

| JSON类型   | Python类型 |
| :--------- | :--------- |
| {}         | dict       |
| []         | list       |
| "string"   | str        |
| 1234.56    | int或float |
| true/false | True/False |
| null       | None       |

### 自定义类型实例的JSON化

很多时候，希望把自定义类的实例 `JSON` 序列化：

```python
import json

class Student(object):
    def __init__(self, name, age, score):
        self.name = name
        self.age = age
        self.score = score

s = Student('Bob', 20, 88)
print(json.dumps(s))

# 结果
Traceback (most recent call last):
  ...
TypeError: <__main__.Student object at 0x10603cc50> is not JSON serializable
```

错误的原因是当前 `Student` 对象不是一个可序列化为 `JSON` 的对象。

`dumps()` 方法的参数列表，除了第一个必须的`obj`参数外，还有很多可选参数：

[https://docs.python.org/3/library/json.html#json.dumps](https://docs.python.org/3/library/json.html#json.dumps)

可以通过这些可选参数来定制 `JSON` 序列化。

**前面的 `Student` 实例无法 `JSON` 序列化，是因为默认情况下 `dumps()` 不知道如何将 `Student` 实例变为一个 JSON 的 `{}` 对象**。

可选参数 `default` 就是把任意一个对象变成一个可序列为 `JSON` 的对象，只需要为 `Student` 写一个转换函数：

```python
def student2dict(std):
    return {
        'name': std.name,
        'age': std.age,
        'score': std.score
    }

# Student 实例首先被 student2dict() 函数转换成 dict ，然后再被序列化为 JSON
>>> print(json.dumps(s, default=student2dict))
{"age": 20, "name": "Bob", "score": 88}
```

一种可以将任意类的实例变成可 `JSON` 序列化的方式：

```python
# 利用 lambda 表达式，直接传递实例的 __dict__ 属性
print(json.dumps(s, default=lambda obj: obj.__dict__))
```

同样，如果要把 `JSON` 反序列化为一个实例，`loads()` 方法首先转换出一个 `dict` 对象；

然后，我们传入的 `object_hook` 函数负责把 `dict` 转换为实例：

```python
def dict2student(d):
    return Student(d['name'], d['age'], d['score'])

>>> json_str = '{"age": 20, "score": 88, "name": "Bob"}'
>>> print(json.loads(json_str, object_hook=dict2student))
<__main__.Student object at 0x10cd3c190>
```